In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

In [ ]:
def import_file(file_path: str, **kwargs) -> pd.DataFrame:
    try:
        file_extension = file_path.split(".")[-1].lower()

        if file_extension == "csv":
            df = pd.read_csv(file_path, **kwargs)
        elif file_extension == "xlsx":
            df = pd.read_excel(file_path, **kwargs)
        elif file_extension == "parquet":
            df = pd.read_parquet(file_path, **kwargs)
        else:
            raise ValueError(f"Unsupported file extension: {file_extension}")

        return df

    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
        raise
    except pd.errors.ParserError as e:
        print(f"Error: Parsing error for file {file_path} - {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occured: {str(e)}")
        raise


def define_expected_default_rate():
    data = {
        "CompositeRate": ["A", "B1", "B2", "B3", "B4", "C1", "C2", "C3"],
        "corporate": [0.0001, 0.0025, 0.0025, 0.0075, 0.0125, 0.025, 0.075, 0.2],
        "sme": [0.0001, 0.0075, 0.0075, 0.0125, 0.0175, 0.025, 0.075, 0.2],
        "rsme": [0.0001, 0.0075, 0.0075, 0.0125, 0.025, 0.04, 0.075, 0.4],
    }

    expected_default_df = pd.DataFrame(data)

    return expected_default_df


def map_expected_default_rate(
    df: pd.DataFrame, expected_default_rate_df: pd.DataFrame, portfolio: str
) -> pd.DataFrame:
    
    port = portfolio.lower()
    
    df_1 = df.merge(
        expected_default_rate_df[["CompositeRate", f"{port}"]],
        how="left",
        on=["CompositeRate"],
    ).rename(columns={f"{port}": "ExpectedDR"})

    return df_1


def hosmer_lemeshow_test(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values('CompositeRate')

    df["ExpectedEvent"] = df["ExpectedDR"] * (df["Good"] + df["Bad"])
    hl_stat = np.sum(((df["Bad"] - df["ExpectedEvent"]) ** 2) / (df["ExpectedEvent"] * (1 - df["ExpectedDR"])))
    dof = df.shape[0]
    
    p_value = 1 - sp.stats.chi2.cdf(hl_stat, dof)
    
    return hl_stat, p_value

In [ ]:
corp_default_rate_df = import_file("../data/processed/03_rating_summary/corporate_rating_summary.parquet")
sme_default_rate_df = import_file("../data/processed/03_rating_summary/sme_rating_summary.parquet")
rsme_default_rate_df = import_file("../data/processed/03_rating_summary/rsme_rating_summary.parquet")
expected_default_rate_df = define_expected_default_rate()

In [ ]:
base_corporate_df = map_expected_default_rate(corp_default_rate_df, expected_default_rate_df, "corporate")
base_sme_df = map_expected_default_rate(sme_default_rate_df, expected_default_rate_df, "sme")
base_rsme_df = map_expected_default_rate(rsme_default_rate_df, expected_default_rate_df, "rsme")

In [ ]:
print("Corporate Portfolio: " + str(hosmer_lemeshow_test(base_corporate_df)))
print("SMEs Portfolio: " + str(hosmer_lemeshow_test(base_sme_df)))
print("RSME Portfolio: " + str(hosmer_lemeshow_test(base_rsme_df)))